In [13]:
import random
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [14]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

In [15]:
SEED = 1038893

fix_random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

root = "../../data/ml-25m"

Device: cuda


In [16]:
## Data Acquisition
def get_data_from_csv(file: str, nrows=None):
    if nrows:
        df = pd.read_csv(f"{root}/{file}", nrows=nrows)
    else:
        df = pd.read_csv(f"{root}/{file}")
    print(f"Loaded ml-25m data: {root}/{file}")

    return df

In [17]:
def get_ratings_from_csv(nrows=None):
    path = "ratings.csv"

    data = get_data_from_csv(path, nrows)
    # data = get_data_from_csv(path)

    data.drop("timestamp", axis=1, inplace=True)
    # todo: drop user id
    #data.drop("userId", axis=1, inplace=True)

    return data

In [18]:
def get_tag_relevances_from_csv(nrows=None):
    path = "genome-scores.csv"

    return get_data_from_csv(path, nrows)
genome_scores_path = "genome-scores.csv"
ratings_path = "ratings.csv"
ratings = get_ratings_from_csv()
print(ratings)
genome_scores = get_tag_relevances_from_csv()
print(genome_scores)
# films = get_data_from_csv(f"{root}/{ratings}")[]

Loaded ml-25m data: ../../data/ml-25m/ratings.csv
          userId  movieId  rating
0              1      296     5.0
1              1      306     3.5
2              1      307     5.0
3              1      665     5.0
4              1      899     3.5
...          ...      ...     ...
25000090  162541    50872     4.5
25000091  162541    55768     2.5
25000092  162541    56176     2.0
25000093  162541    58559     4.0
25000094  162541    63876     5.0

[25000095 rows x 3 columns]
Loaded ml-25m data: ../../data/ml-25m/genome-scores.csv
          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 colum

In [19]:
# DATA VISUALIZATION

            # # Fill in missing values with zeros
            # X.fillna(0, inplace=True)

# FARE TEST CON AVG, STD_DEV
def addColumnOperation(ratings,X):
     # Compute the mean rating for each user
     count_rating = ratings.groupby('movieId', as_index=False)['rating'].count()
     std= ratings.groupby('movieId', as_index=False)['rating'].std()
     std.fillna(0, inplace=True)
     min_ratings= ratings.groupby('movieId', as_index=False)['rating'].min()
     max_ratings= ratings.groupby('movieId', as_index=False)['rating'].max()
     median= ratings.groupby('movieId', as_index=False)['rating'].median()
     operation = pd.DataFrame({'movieId':count_rating['movieId'],'count_rating': count_rating['rating'], 'std': std['rating'], 'min': min_ratings['rating'], 'max': max_ratings['rating'], 'median': median['rating']}) 
     X = pd.merge(X, operation, on='movieId')
     X.drop("movieId", axis=1, inplace=True)
     return X

In [20]:
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabnet.augmentations import RegressionSMOTE
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
           

class TabNet:
    def __init__(self, ratings, relevance, seed=42,width_values = 8, steps = 3, learning_rate = 2e-2):
        self.aug = RegressionSMOTE(p=0.2)
        #! df['rating'] = df['rating'].astype('float16')

        # Reduce genome-score size

        # Pivot the relevance DataFrame to create a matrix of tag relevance scores for each movie
        relevance_matrix = relevance.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)
        # Merge the ratings and relevance data
        mean_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()
        X = mean_ratings.merge(relevance_matrix, on='movieId')
        #X = addColumnOperation(ratings,X)
        X.columns = X.columns.astype(str)
        ratings = None  
        train = X
        # mescolare le righe del DataFrame
        X = X.sample(frac=1,random_state = seed).reset_index(drop=True)
        
        if "Set" not in train.columns:
            train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

        features = [ col for col in train.columns if col not in ["rating", "Set"]]
        target = "rating"
        
        train_indices = train[train.Set=="train"].index
        valid_indices = train[train.Set=="valid"].index
        test_indices = train[train.Set=="test"].index

        self.X_train = train[features].values[train_indices]
        self.y_train = train[target].values[train_indices].reshape(-1, 1)

        self.X_valid = train[features].values[valid_indices]
        self.y_valid = train[target].values[valid_indices].reshape(-1, 1)

        self.X_test = train[features].values[test_indices]
        self.y_test = train[target].values[test_indices].reshape(-1, 1)
        
        if torch.cuda.is_available():
            self.model = TabNetRegressor(n_d = width_values, n_a = width_values , n_steps = steps, optimizer_params = dict(lr=learning_rate), seed=seed, device_name="cuda")  
        else:
            self.model = TabNetRegressor(n_d = width_values, n_a = width_values , n_steps = steps, optimizer_params = dict(lr=learning_rate), seed=seed)  


    def train(self,max_epochs = 150,batchsize = 1024):
        self.model.fit(
            X_train=self.X_train, y_train=self.y_train,
            eval_set=[(self.X_train,self.y_train), (self.X_valid, self.y_valid)],
            eval_name=['train', 'valid'],
            eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
            max_epochs=max_epochs,
            patience=20,
            batch_size=batchsize, virtual_batch_size=1024,
            num_workers=0,
            drop_last=False,
            augmentations=self.aug, #aug
        ) 

    def test(self):
        # Predict the labels of the test set: y_pred
        y_pred = self.model.predict(self.X_test)

        # Compute the mean squared error
        mse = mean_squared_error(self.y_test, y_pred)
        rmse = mean_squared_error(self.y_test, y_pred, squared=False)
        r2 = r2_score(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)

        print(f"MSE: {mse} RMSE: {rmse} R2: {r2} MAE: {mae}")
        print("=====================================")
        return r2,self.model
    
    def load(self,model):
        self.model =TabNetRegressor()
        self.model.load_model(model)
    
    def save(self,root,name):
        self.model.save_model(f"{root}/{name}.pt")


In [21]:
model = TabNet(ratings, genome_scores)

c:\Users\Daniele\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [22]:
import itertools

batchsize = [2048]
width = [8,16,32]
steps = [3,5,7]
learning_rate = [2e-2,1e-2,5e-3]
max_epochs = [70,120,150,210]

best_model_params = None 
best_r2 = 0

total_iterations = len(batchsize) * len(width) * len(steps) * len(learning_rate) * len(max_epochs)
current_iteration = 0

for batchsize,width,steps,learning_rate,max_epochs in itertools.product(batchsize,width,steps,learning_rate,max_epochs):
    current_iteration += 1
    print(f"[{current_iteration}/{total_iterations}] START => batchsize: {batchsize} width: {width} steps: {steps} learning_rate: {learning_rate} max_epochs: {max_epochs}")
    model = TabNet(ratings, genome_scores,width_values = width, steps = steps, learning_rate = learning_rate)
    model.train(max_epochs = max_epochs,batchsize = batchsize)
    r2score, instance = model.test()
    if r2score > best_r2:
        best_r2 = r2score
        best_model_params = f'{batchsize}_{width}_{steps}_{learning_rate}_{max_epochs}'
        model.save("model",best_model_params)
        print(f"New best model: {best_model_params} with r2: {best_r2}")

model = TabNet(ratings,genome_scores)
print(f'Best model SCORE: {best_r2}')
model.load(f"model/{best_model_params}.pt.zip")
print(model.test())



[1/108] START => batchsize: 2048 width: 8 steps: 3 learning_rate: 0.02 max_epochs: 70


c:\Users\Daniele\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 9.57511 | train_rmsle: 0.91517 | train_mae: 2.62767 | train_rmse: 2.67362 | train_mse: 7.14826 | valid_rmsle: 0.91812 | valid_mae: 2.63432 | valid_rmse: 2.67924 | valid_mse: 7.17831 |  0:00:03s
epoch 1  | loss: 3.78898 | train_rmsle: 0.22977 | train_mae: 1.60089 | train_rmse: 1.67549 | train_mse: 2.80727 | valid_rmsle: 0.23051 | valid_mae: 1.60573 | valid_rmse: 1.67947 | valid_mse: 2.82063 |  0:00:03s
epoch 2  | loss: 1.52475 | train_rmsle: 0.07568 | train_mae: 0.97686 | train_rmse: 1.07345 | train_mse: 1.1523  | valid_rmsle: 0.07562 | valid_mae: 0.97727 | valid_rmse: 1.07505 | valid_mse: 1.15573 |  0:00:04s
epoch 3  | loss: 0.75498 | train_rmsle: 0.13202 | train_mae: 1.26107 | train_rmse: 1.35013 | train_mse: 1.82284 | valid_rmsle: 0.13232 | valid_mae: 1.26329 | valid_rmse: 1.35319 | valid_mse: 1.83112 |  0:00:05s
epoch 4  | loss: 0.34824 | train_rmsle: 0.06304 | train_mae: 0.89703 | train_rmse: 0.99306 | train_mse: 0.98616 | valid_rmsle: 0.06296 | valid_mae: 0.89713 

c:\Users\Daniele\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.02395427708844633 RMSE: 0.15477169343405897 R2: 0.8939636453573545 MAE: 0.12396044094449743
Successfully saved model at model/2048_8_3_0.02_70.pt.zip
New best model: 2048_8_3_0.02_70 with r2: 0.8939636453573545
[2/108] START => batchsize: 2048 width: 8 steps: 3 learning_rate: 0.02 max_epochs: 120


...

epoch 0  | loss: 8.89223 | train_rmsle: 0.60047 | train_mae: 2.29687 | train_rmse: 2.34475 | train_mse: 5.49783 | valid_rmsle: 0.60121 | valid_mae: 2.30171 | valid_rmse: 2.34838 | valid_mse: 5.51489 |  0:00:01s
epoch 1  | loss: 5.73513 | train_rmsle: 0.25085 | train_mae: 1.66324 | train_rmse: 1.7305  | train_mse: 2.99462 | valid_rmsle: 0.25184 | valid_mae: 1.66868 | valid_rmse: 1.73506 | valid_mse: 3.01042 |  0:00:02s
epoch 2  | loss: 4.28118 | train_rmsle: 0.10197 | train_mae: 1.12825 | train_rmse: 1.2152  | train_mse: 1.47672 | valid_rmsle: 0.10223 | valid_mae: 1.13014 | valid_rmse: 1.21818 | valid_mse: 1.48396 |  0:00:03s
epoch 3  | loss: 3.60404 | train_rmsle: 0.05442 | train_mae: 0.83676 | train_rmse: 0.93204 | train_mse: 0.86869 | valid_rmsle: 0.05458 | valid_mae: 0.8373  | valid_rmse: 0.93538 | valid_mse: 0.87494 |  0:00:04s
epoch 4  | loss: 2.94899 | train_rmsle: 0.02571 | train_mae: 0.57171 | train_rmse: 0.66298 | train_mse: 0.43954 | valid_rmsle: 0.02546 | valid_mae: 0.57167 

c:\Users\Daniele\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.0346047252985036 RMSE: 0.18602345362481473 R2: 0.8468182149469555 MAE: 0.14124263040808283


c:\Users\Daniele\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Best model SCORE: 0.9647752236735118
MSE: 0.007957496579413888 RMSE: 0.08920480132489443 R2: 0.9647752288141751 MAE: 0.06890399590397187
(0.9647752288141751, TabNetRegressor(n_d=8, n_a=8, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=1129, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1))
